# <center> **Dense Optical Flow** </center>

Dense optical flow attempts to compute the optical flow vector for every pixel of each frame. While such computation may be slower, it gives a more accurate result and a denser result suitable for applications such as learning structure from motion and video segmentation.

In [ ]:
# Libraries
import numpy as np
import cv2
from scipy.signal import fftconvolve
from matplotlib import pyplot as plt

## **Image Gradient**

* Image is a matrix of pixel values representing various intensity level values.
* A pixel is the building block of an image.
* The gradient can be defined as the change in the direction of the intensity level of an image.

In [ ]:
# Filters
# Sobel Filter
Sobel_filter_x = np.array([[1, 0, -1],
                           [2, 0, -2],
                           [1, 0, -1]])

Sobel_filter_y =np.array([[ 1,  2,  1],
                          [ 0,  0,  0],
                          [-1, -2, -1]])

# Scharr Filter
Scharr_filter_x = np.array([[ -3, 0,  3],
                            [-10, 0, 10],
                            [ -3, 0,  3]])

Scharr_filter_y = np.array([[ 3, 10,  3],
                            [ 0, 0 ,  0],
                            [-3, 10, -3]])

# Prewitt Filter
Prewitt_filter_x = np.array([[1, 0, -1],
                             [1, 0, -1],
                             [1, 0, -1]])

Prewitt_filter_y = np.array([[ 1,  1,  1],
                             [ 0,  0,  0],
                             [-1, -1, -1]])

# Roberts Filter
Roberts_filter_x = np.array([[1,  0],
                             [0, -1]])

Roberts_filter_y = np.array([[ 0, 1],
                             [-1, 0]])

# Averaging Filter
Averaging_filter_x = np.array([[-0.25, 0.25],
                               [-0.25, 0.25]])

Averaging_filter_y = np.array([[-0.25, -0.25],
                               [ 0.25, 0.25]])

Averaging_filter_t = np.array([[0.25, 0.25],
                               [0.25, 0.25]])

In [ ]:
def image_gradient(img, kernel_x, kernel_y):
    Ix = fftconvolve(in1=img, in2=kernel_x, mode='same')
    Iy = fftconvolve(in1=img, in2=kernel_y, mode='same')
    return Ix, Iy

def image_gradient2img(img1, img2, kernel_x, kernel_y, kernel_t):
    Ix = fftconvolve(in1=img1, in2=kernel_x, mode='same')
    Iy = fftconvolve(in1=img1, in2=kernel_y, mode='same')
    It = fftconvolve(in1=img1, in2=kernel_t, mode='same') + fftconvolve(in1=img2, in2=-kernel_t, mode='same')
    return Ix, Iy, It

# **Horn-Schunck method**

### **Horn-Schunck Function**

In [ ]:
# Function: Horn-Schunck Optical Flow method
# PI: Previous image (np.ndarray)
# LI: Last image (np.ndarray)
# corner_list: List corners
# window_size: Window size (tuple)
# Return: vx, vy, corner_list
def Horn_Schunck_Optical_Flow(PI, LI, alpha, window_size=(3, 3)):
    shape = np.shape(PI)

    # Median blur: Calculate average of neighbors
    LI_blur = cv2.blur(LI,window_size)

    # First Derivative in X, Y, XY direction
    Ix = Iy = It = np.zeros_like(PI)

    Ix[:, :-1] = (LI[:, 1:] - LI[:, :-1]) / 2.0
    Iy[:-1, :] = (LI[1:, :] - LI[:-1, :]) / 2.0
    It[:, :] = LI - PI

    # Creating vx, vy
    vx = vy = np.zeros(shape)

    # Plot features
    err_val = (Ix * uAvg + Ix * vAvg + It) / (alpha ** 2 + Ix ** 2 + Iy ** 2)

    return vx, vy

## **Plot image - Plot video**

In [ ]:
# Function: Plot Optical Flow
# PI: Previous image (np.ndarray)
# LI: Last image (np.ndarray)
# vx: vx (np.ndarray)
# vy: vy (np.ndarray)
# fig_size: Figure size (tuple)
# threshold: Threshold
# Result: Plot vector on the previous image
def plot_image_optical_flow(PI, LI, vx, vy, fig_size=(10, 10), threshold=0.3):
    shape = np.shape(PI)

    # Create some random colors
    color = np.random.randint(0,255,(1,3))
    fig = plt.figure()
    fig.set_figwidth(fig_size[0])
    fig.set_figheight(fig_size[1])

    # Plotting the vectors on the image
    plt.subplot(3, 1, 2)
    plt.title('Image 2')
    plt.imshow(LI)

    plt.subplot(3, 1, 3)
    plt.title('Plotting vector')
    plt.imshow(LI)

    for y in range(1, shape[0] - 1):
        for x in range(1, shape[1] - 1):
            if abs(vx[y][x]) > threshold or abs(vy[y][x]) > threshold: # setting the threshold to plot the vectors
                cv2.circle(PI, (x, y), 3, color[0].tolist(), -1)
                plt.arrow(x, y, vy[y][x], vx[y][x], head_width = 5, head_length = 5, color = 'g')
    
    plt.subplot(3, 1, 1)
    plt.title('Image 1')
    plt.imshow(PI)

In [ ]:
# Function: Plot Optical Flow
# PI: Previous image (np.ndarray)
# LI: Last image (np.ndarray)
# vx: vx (np.ndarray)
# vy: vy (np.ndarray)
# fig_size: Figure size (tuple)
# threshold: Threshold
# Result: Plot vector on the previous image
def plot_video_optical_flow(frame, mask, vx, vy, threshold=0.3, title="image", color=(0, 255, 0)):
    shape = np.shape(frame)

    for y in range(1, shape[0] - 1):
        for x in range(1, shape[1] - 1):
            if abs(vx[y][x]) > threshold or abs(vy[y][x]) > threshold: # setting the threshold to plot the vectors
                # cv2.circle(frame, (x, y), 5, color[i].tolist(), -1)
                mask = cv2.line(mask, (x, y), (int(x + vx[y][x]), int(y + vy[y][x])), 
                                color = color[i], thickness = 3)
        
    img = cv2.add(frame, mask)
    cv2.imshow(title, img)
    return mask

### **Test Horn-Schunck function with two frames**

### **Test Horn-Schunck function with video**

# **Compare to OpenCV**

# **Different from Lucas-Kanade method**